In [153]:
import numpy as np
from sklearn.decomposition import NMF
import pandas as pd
import pickle
import requests 
import re
from bs4 import BeautifulSoup

In [3]:
movie_mapping = dict(zip(movies.movieId, movies.title))

In [4]:
imdb_mapping = dict(zip(links.movieId, links.imdbId))

In [6]:
ratings['movie_title']=ratings['movieId'].map(movie_mapping)
ratings['imdb']=ratings['movieId'].map(imdb_mapping)
ratings.head()

,userId,movieId,rating,timestamp,movie_title,imdb
0,1,1,4.0,964982703,Toy Story (1995),114709
1,1,3,4.0,964981247,Grumpier Old Men (1995),113228
2,1,6,4.0,964982224,Heat (1995),113277
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),114369
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",114814


In [8]:
#dict with movie title/index
dic_imdb={}
for num,title in enumerate(ratings.movie_title):
    dic_imdb[title]=ratings.imdb[num]
pickle.dump(dic_imdb, open("dic_imdb.pkl", "wb")) 

In [17]:
mx=pd.pivot_table(ratings, values="rating", index="userId", columns="movie_title")

In [20]:
mean=mx.mean().mean()
print(mean)
mx=mx.fillna(mean)

3.2623883140510226


In [21]:
mx.head()

movie_title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,...,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,4.000000,3.262388
2,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,...,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388
3,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,...,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388
4,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,...,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388
5,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,...,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388,3.262388


In [22]:
pickle.dump(mx, open("mx.pkl", "wb"))

In [23]:
mx_array=mx.values

In [24]:
#dict with movie title/index
dic={}
for i,j in enumerate(mx.columns):
    dic[j]=i

In [25]:
main_moive_list=list(dic.keys())
pickle.dump(main_moive_list, open("main_moive_list.pkl", "wb")) 

In [26]:
pickle.dump(dic, open("dic.pkl", "wb")) 

In [27]:
#create a model and set the hyperparameters
model = NMF(n_components=20, init='random', max_iter=1000,random_state=10)
model.fit(mx_array)

/home/daria/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


NMF(init='random', max_iter=1000, n_components=20, random_state=10)

In [28]:
pickle.dump(model, open("nmf.pkl", "wb")) 

In [29]:
Q = model.components_  # movie matrix, where each row of Q represents the strength of the associations between an item(movie) and the latent features
P = model.transform(mx_array)  # user-matrix, where each row of P represents the strength of the associations between a user and the latent features
pred = np.dot(P, Q) # The reconstructed matrix: predictions

In [33]:
pickle.dump(Q, open("nmf_Q.pkl", "wb"))

In [34]:
new_user = np.full(shape=(1,mx.shape[1]), fill_value=mean); new_user, new_user.shape

(array([[3.26238831, 3.26238831, 3.26238831, ..., 3.26238831, 3.26238831,
         3.26238831]]),
 (1, 9719))

In [35]:
movie_list=["'71 (2014)","'Hellboy': The Seeds of Creation (2004)"]
rating_list=['5','4']

In [36]:
for i,m in enumerate(movie_list):    
    new_user[0][dic[m]] = rating_list[i]

In [37]:
new_user

array([[5.        , 4.        , 3.26238831, ..., 3.26238831, 3.26238831,
        3.26238831]])

In [46]:
user_P = model.transform(new_user) 
actual_recommendations = np.dot(user_P, Q)

In [47]:
np.sort(actual_recommendations)[0][::-1][:5] #top 5 probs

array([3.58002249, 3.45473544, 3.3992052 , 3.39439532, 3.39092984])

In [48]:
ind=np.argsort(actual_recommendations)[0][::-1][:5] #top 5 indexes containing movie titles
ind

array([7593, 3158, 3279, 7680, 6865])

In [41]:
recommended_movies=pd.Series(actual_recommendations[0], index=mx.columns)
recommended_movies

movie_title
'71 (2014)                                   3.263321
'Hellboy': The Seeds of Creation (2004)      3.262368
'Round Midnight (1986)                       3.263299
'Salem's Lot (2004)                          3.268409
'Til There Was You (1997)                    3.267686
                                               ...   
eXistenZ (1999)                              3.270705
xXx (2002)                                   3.256380
xXx: State of the Union (2005)               3.266654
¡Three Amigos! (1986)                        3.256400
À nous la liberté (Freedom for Us) (1931)    3.260792
Length: 9719, dtype: float64

In [52]:
#create user df
user_df=recommended_movies.to_frame('predictions').reset_index()

In [53]:
user_df

,movie_title,predictions
0,'71 (2014),3.263321
1,'Hellboy': The Seeds of Creation (2004),3.262368
2,'Round Midnight (1986),3.263299
3,'Salem's Lot (2004),3.268409
4,'Til There Was You (1997),3.267686
...,...,...
9714,eXistenZ (1999),3.270705
9715,xXx (2002),3.256380
9716,xXx: State of the Union (2005),3.266654
9717,¡Three Amigos! (1986),3.256400


In [44]:
import re
def year(y):
    """extracts year from the movies title"""
    year=re.findall("\(([0-9]+)\)",y)
    try:
        year=year[0]
    except:
        year=0
    return int(year)

In [54]:
user_df['year']=user_df['movie_title'].apply(year)
user_df=user_df.set_index("movie_title")
user_df

,predictions,year
movie_title,,
'71 (2014),3.263321,2014
'Hellboy': The Seeds of Creation (2004),3.262368,2004
'Round Midnight (1986),3.263299,1986
'Salem's Lot (2004),3.268409,2004
'Til There Was You (1997),3.267686,1997
...,...,...
eXistenZ (1999),3.270705,1999
xXx (2002),3.256380,2002
xXx: State of the Union (2005),3.266654,2005


In [55]:
#removing seen movies
user_df=user_df.loc[~user_df.index.isin(movie_list)] 
user_df

,predictions,year
movie_title,,
'Round Midnight (1986),3.263299,1986
'Salem's Lot (2004),3.268409,2004
'Til There Was You (1997),3.267686,1997
'Tis the Season for Love (2015),3.258395,2015
"'burbs, The (1989)",3.268256,1989
...,...,...
eXistenZ (1999),3.270705,1999
xXx (2002),3.256380,2002
xXx: State of the Union (2005),3.266654,2005


In [282]:
#10 best movies in the last 10 years
choice=user_df.reset_index().sort_values(by=['predictions']).groupby('year').max()[-10:]['movie_title'].values

In [283]:
#randomly pick 5 of out those 10
new=list(np.random.choice(choice,5, replace=False))
new

['Zombeavers (2014)',
 '[REC]² (2009)',
 "Won't You Be My Neighbor? (2018)",
 'You Will Meet a Tall Dark Stranger (2010)',
 'Zoom (2015)']

In [145]:
#all movies sorted by predictions
all=list(user_df.sort_values(by='predictions', ascending=False)[:5].index)
all

['Shawshank Redemption, The (1994)',
 'Forrest Gump (1994)',
 'Fugitive, The (1993)',
 'Silence of the Lambs, The (1991)',
 'Pulp Fiction (1994)']

In [284]:
def get_link(x):
    """returns poster link"""
    x=str(x)
    if len(x)==7:
        url=f"https://www.omdbapi.com/?i=tt{x}&apikey=e815a9b1"
    if len(x)==6:
        url=f"https://www.omdbapi.com/?i=tt0{x}&apikey=e815a9b1"
    else:
        url=f"https://www.omdbapi.com/?i=tt00{x}&apikey=e815a9b1"
#    print(url)
    page = requests.get(url).json()
#    print(page)
    link=page.get("Poster")

        
    return link


In [286]:
def get_pics(movies):
            "returns list of posters of recommended movies "
            pics=[]
            for i in movies:
                ID=dic_imdb[i]
                print(ID)
                link=get_link(ID)
                print(link)
                pics.append(link)
            return pics